# LIBRARIES

In [2]:
import pandas as pd
import numpy as np
from pkg.constants import *
from pkg.greedy import greedy
from pkg.evaluation import Evaluation
from pkg.utils import *
from pkg.random_search import random_search
from pkg.local_search import local_search
from pkg.neighborhood import *
from itertools import permutations
from random import shuffle


# GENERACIÓN DE SEMILLAS
Las semillas deben guardarse para poder repetir las ejecuciones y comparar resultados

In [3]:
# seeds = np.random.randint(0,99999,SEEDS)
# print(seeds)

# Semillas adri
seeds = np.array([12345, 54321, 12221, 22211, 22212])
print(seeds)

[12345 54321 12221 22211 22212]


# CREACION OBJETO EVALUATION

In [4]:
evaluation = Evaluation()

# SOLUCIÓN GREEDY

## LECTURA DE DATOS

In [5]:
deltas_df   = pd.read_csv(DELTAS_5M_PATH) 
deltas_matrix   = deltas_df.to_numpy()

## GENERA SOLUCION 

In [6]:
initial_state = deltas_matrix[0,:]
greedy_sol = greedy(initial_state)
greedy_sol.sum()

220.0

## EVALUA SOLUCION

In [7]:
# 574.5513854177129
cost_greedy_solution = evaluation.evaluate(greedy_sol)

# BUSQUEDA ALEATORIA

### ALGORITMO

In [ ]:
best_solution_per_seed = np.empty((SEEDS, TOTAL_STATIONS))


for i in range(SEEDS):
    np.random.seed(seeds[i])
    best_solution_per_seed[i,:] = random_search(evaluation = evaluation)


In [ ]:
print(best_solution_per_seed)
print(evaluation.evaluate(best_solution_per_seed[0,:]))
print(evaluation.evaluate(best_solution_per_seed[1,:]))
print(evaluation.evaluate(best_solution_per_seed[2,:]))
print(evaluation.evaluate(best_solution_per_seed[3,:]))
print(evaluation.evaluate(best_solution_per_seed[4,:]))


# BUSQUEDA LOCAL: EL PRIMER MEJOR VECINO

## Generación soluciones iniciales

In [11]:
seeds

array([12345, 54321, 12221, 22211, 22212])

## Algoritmo


In [1]:
for i in range(SEEDS):

    np.random.seed(seeds[i])

    sol = local_search(
        slots_to_move = 1,
        evaluation    = evaluation
    )
    
    print(sol)
    print(evaluation.evaluate(sol))

    

NameError: name 'SEEDS' is not defined

# ENFRIAMIENTO SIMULADO

In [14]:
# parametros a ajustar:
slots = 1

# k_max: maximo numero iteraciones: en el entorno de 80
# condicion de parada
k_max = 80

# L(T): condicion enfriamiento. Numero fijo para cada iteracion
# Se enfria la temperatura al final de una iteracion
# La iteracion finaliza al visitar todos los vecinos
L = 20


# Esquema de enfriamiento: Cauchy
# tk = t0/(1+k)

# mu: tanto por uno [0,1], indica la probabilidad de aceptar una solucion peor que la inicial
mu = 0.2

# fi: probabilidad de aceptar una solucion un mu por 1 peor que la inicial
fi = 0.2

# Temperatura inicial
t0 = mu * cost_greedy_solution / (- np.log(fi))

# Temperatura final
tf = t0 / (1 + k_max)



In [19]:
def simulated_an(t0,L,tf,slots,evaluation):

    t = t0

    current_solution      = generate_random_solution()
    cost_current_solution = evaluation.evaluate(current_solution)

    neighbors = Neighborhood(
        current_solution = current_solution,
        slots            = slots
    )

    k = 0

    while t >= tf:
        for cont in range(L):

            candidate_solution      = neighbors.get_neighbor_sa()
            cost_candidate_solution = evaluation.evaluate(candidate_solution)

            cost_difference = cost_candidate_solution - cost_current_solution

            if cost_difference < 0 or \
               np.random.rand() < np.exp( (-cost_difference) / t ):

                current_solution      = candidate_solution
                cost_current_solution = cost_candidate_solution 

                neighbors = Neighborhood(
                    current_solution = current_solution,
                    slots            = slots
                )
            
        t = t0 / (1 + k)
        k += 1

    return current_solution, cost_current_solution
                

In [20]:
for i in range(SEEDS):
    np.random.seed(seeds[i])
    sa_solution, cost_sa = simulated_an(t0,L,tf,slots,evaluation)
    print(sa_solution)
    print(cost_sa)



[15. 12. 16. 18. 20. 14. 11. 12. 10. 19. 12. 23.  2.  2. 20. 13.]
384.2487482445857
[14. 12. 19. 19. 20. 14. 12. 11. 12. 16. 17. 25.  2.  6. 14. 10.]
345.9125412943197
[17. 12. 19. 16. 20. 14.  8. 12. 12. 13. 12. 24.  3.  7. 18. 14.]
361.6959227450305
[14. 10. 21. 14. 20. 14. 12. 11. 12. 14. 12. 21.  2. 19.  7. 13.]
409.7041593111751
[18. 12. 14. 17. 20. 14. 13. 12. 10. 16. 18. 34.  2. 17.  5. 11.]
280.8809511306812


# ANOTACIONES DIA 21/03/2022

Dia 4: defensa de la práctica
la bsuqueda  tabu deberá tardar más o menos lo que tarda el enfriamiento simulado.
No tenemos en cuenta si los vecinos se repiten o no (no lo combprobamos.)
velocidad de movimiento: 5 (nos dice cuantas posibilidades hay)
matriz de frecuencias: como se hace??? 
greedy: queremos hacer algo inversamente prorporcional a lo que ha salido (hacemos la inversa de cada casilla de la matriz de frecuencias???) y luego normalizo (?)
ruleta de probabilidad inversa -> buscar libreria, que seguro que está hecho en algun lado. Si no, está el pseudo en la práctica.

la condicion de parada: que sea mayor o igual al numero de iteraciones 
generar vecinos tiene la restriccion tabu
cogemos el algoritmo de busqueda local y vmos añadiendo trocitos
la parte que suele generar más dudas suele ser la parte del greedy


granular: tamaño de movimiento de huecos entre estaciones